In [1]:
# %load ImportsDefinitions.py
#!/usr/bin/env python

# In[1]:


#Imports
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re
import joblib 
import pickle
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import time
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree

from sklearn.svm import LinearSVC


import seaborn as sns

#accuracy_score(labels_test,pred)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
#accuracy_score(labels_test,pred)
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from yellowbrick.text import FreqDistVisualizer
from yellowbrick.datasets import load_hobbies
from sklearn.model_selection import TimeSeriesSplit



from yellowbrick.classifier import ClassificationReport
from yellowbrick.datasets import load_occupancy
from yellowbrick.text import FreqDistVisualizer
from yellowbrick.datasets import load_hobbies


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('vader_lexicon')


# In[1]:


#Ορισμός συναρτήσεων για να ελέξγχουμε τον καλύτερο vectorizer
def tfidf_test_simple(X_train,X_test,y_train,y_test,token_izer):
    if (token_izer=='1'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor)
    elif (token_izer=='2'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor_imdb)
    else:
        tfvect= TfidfVectorizer()
    tfidf_train = tfvect.fit_transform(X_train)
    tfidf_test = tfvect.transform(X_test)
    nb=MultinomialNB()
    #train the model and timing it
    #kanoume prediction gia to x_test_dtm
    
    # cross val score/ predict
    cvec_score = cross_val_score(nb, tfidf_train, y_train, cv=4 )
    feature_names = tfvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("to accuracy tou TFIDF me NB einai {}".format(cvec_score.mean()))
    
    visualizer = FreqDistVisualizer(features=feature_names, orient='h')
    visualizer.fit(tfidf_train)
    visualizer.poof()
    
    return cvec_score.mean()

def countvect_test_simple(X_train,X_test,y_train,y_test,token_izer):
    if (token_izer=='1'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor)
    elif (token_izer=='2'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor_imdb)
    else:
        countvect= CountVectorizer()
    #CountVect
    countvect.fit(X_train)
    #to metatrepoume se dtm sparse matrix
    X_train_dtm=countvect.transform(X_train)
    X_test_dtm=countvect.transform(X_test)
    #Ftiaxnoume Multinomial Naive Bayes modelo
    nb=MultinomialNB()
    #kanoume prediction gia to x_test_dtm
    
    # cross val score/ predict
    cvec_score = cross_val_score(nb, X_train_dtm, y_train, cv=4 )
        
          
    feature_names = countvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("to accuracy tou CountVectorizer me NB einai: {}".format(cvec_score.mean()))
    
    visualizer = FreqDistVisualizer(features=feature_names, orient='h')
    visualizer.fit(X_train_dtm)
    visualizer.poof()
    
    return cvec_score.mean()


# In[9]:


#oρισμός συναρτήσεων για να ελέγχουμε το max_df
def countvect_test_maxdf(X_train,X_test,y_train,y_test,token_izer,maxdf):
    if (token_izer=='1'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor,max_df=maxdf)
    elif (token_izer=='2'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor_imdb,max_df=maxdf)
    else:
        countvect= CountVectorizer(max_df=maxdf)
    X_train_dtm = countvect.fit_transform(X_train)
    X_test_dtm = countvect.transform(X_test)
    nb=MultinomialNB()
    cvec_score = cross_val_score(nb, X_train_dtm, y_train, cv=4 )
   
    #kanoume evaluate ta apotelesmata mas me Logistic Regression
    feature_names = countvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("To accuracy NB me Max df: {} είναι : {} ".format(maxdf,cvec_score))
    return cvec_score.mean()


def tfidf_test_maxdf(X_train,X_test,y_train,y_test,token_izer,maxdf):
    if (token_izer=='1'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor,max_df=maxdf)
    elif (token_izer=='2'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor_imdb,max_df=maxdf)
    else:
        tfvect= TfidfVectorizer(max_df=maxdf)
    tfidf_train = tfvect.fit_transform(X_train)
    tfidf_test = tfvect.transform(X_test)
    nb=MultinomialNB()
    cvec_score = cross_val_score(nb, tfidf_train, y_train, cv=4 )
    
    #kanoume evaluate ta apotelesmata mas me Logistic Regression
    feature_names = tfvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("To accuracy NB me max df: {} είναι : {} ".format(maxdf,cvec_score))
    return cvec_score.mean()


# In[12]:


def countvect_test_ngrams(X_train,X_test,y_train,y_test,token_izer,ngrams):
    if (token_izer=='1'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor,ngram_range=(1,ngrams))
    elif (token_izer=='2'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor_imdb,ngram_range=(1,ngrams))
    else:
        countvect= CountVectorizer(ngram_range=(1,ngrams))
    X_train_dtm = countvect.fit_transform(X_train)
    X_test_dtm = countvect.transform(X_test)
    nb=MultinomialNB()
    cvec_score = cross_val_score(nb, X_train_dtm, y_train, cv=4 )
   
    #kanoume evaluate ta apotelesmata mas me Logistic Regression
    feature_names = countvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("To accuracy NB me {}-ngrams είναι : {} ".format(ngrams,cvec_score))
    return cvec_score.mean()

def tfidf_test_ngrams(X_train,X_test,y_train,y_test,token_izer,ngrams):
    if (token_izer=='1'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor,ngram_range=(1,ngrams))
    elif (token_izer=='2'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor_imdb,ngram_range=(1,ngrams))
    else:
        tfvect= TfidfVectorizer(ngram_range=(1,ngrams))
    tfidf_train = tfvect.fit_transform(X_train)
    tfidf_test = tfvect.transform(X_test)
    nb=MultinomialNB()
    cvec_score = cross_val_score(nb, tfidf_train, y_train, cv=4 )
    
    #kanoume evaluate ta apotelesmata mas me Logistic Regression
    feature_names = tfvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("To accuracy NB me {}-ngrams είναι : {} ".format(ngrams,cvec_score))
    return cvec_score.mean()


# In[ ]:


def countvect_test_maxfeat(X_train,X_test,y_train,y_test,token_izer,maxfeat):  
    if (token_izer=='1'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor,max_features=maxfeat)
    elif (token_izer=='2'):
        countvect= CountVectorizer(tokenizer=tokenizer_preproccessor_imdb,max_features=maxfeat)
    else:
        countvect= CountVectorizer(max_features=maxfeat)
    X_train_dtm = countvect.fit_transform(X_train)
    X_test_dtm = countvect.transform(X_test)
    nb=MultinomialNB()
    cvec_score = cross_val_score(nb, X_train_dtm, y_train, cv=4 )
   
    #kanoume evaluate ta apotelesmata mas me Logistic Regression
    feature_names = countvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("To accuracy NB me max features {} είναι : {} ".format(maxfeat,cvec_score))
    return cvec_score.mean()

def tfidf_test_maxfeat(X_train,X_test,y_train,y_test,token_izer,maxfeat):
    if (token_izer=='1'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor,max_features=maxfeat)
    elif (token_izer=='2'):
        tfvect= TfidfVectorizer(tokenizer=tokenizer_preproccessor_imdb,max_features=maxfeat)
    else:
        tfvect= TfidfVectorizer(max_features=maxfeat)
    tfidf_train = tfvect.fit_transform(X_train)
    tfidf_test = tfvect.transform(X_test)
    nb=MultinomialNB()
    cvec_score = cross_val_score(nb, tfidf_train, y_train, cv=4 )
    
    #kanoume evaluate ta apotelesmata mas me Logistic Regression
    feature_names = tfvect.get_feature_names()
    print("Number of features: {}".format(len(feature_names)))
    print("To accuracy NB me max featurues : {} είναι : {} ".format(maxfeat,cvec_score))
    return cvec_score.mean()


# In[4]:


def tokenizer_preproccessor(text):
    stop = set(stopwords.words('english'))
    #kanoume preprocessing ta dedomena mas
    word_tokens=word_tokenize(text.lower()) #kanoume tokenize
    #print("Έχουμε " + str(len(word_tokens))+ " tokens")
    #filtered_word_tokens = [word for word in word_tokens if word not in stop] #svinoume ta stopwords
    #print("Αφού αφαιρέσαμε τα stopwords, έχουμε τελικά " + str(len(filtered_word_tokens)) + " tokens")
    #print(filtered_word_tokens)
    filtered_word_tokens = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in word_tokens]  #svinoume ta punctuations
    #filtered_word_tokens = [x for x in filtered_word_tokens if x not in ['film','movie']]  #svinoume ta punctuations    
    filtered_word_tokens = [word for word in filtered_word_tokens if len(word)>1] #svinoume tis mikres lekseis
    #print("Αφού αφαιρέσαμε τα σημεία στίξης και τις μικρές λέξεις, έχουμε τελικά " + str(len(filtered_word_tokens)) + " tokens")
    #print(filtered_word_tokens)
    tagged_filtered_tokens=nltk.pos_tag(filtered_word_tokens) #kanoume pos tag tis lekseis gia syntaktiki analysi
    #print(tagged_filtered_tokens)
            #ftiaxnoume ta pos tags wste na mpoun san input sto sentisynset
    #epeidh ta dedomena tou postag ginontai tuples, ta metatrepoume se lista
    newtags=[] #lista me ta nea tags kai idio counter me ta tagged words
    for tag in tagged_filtered_tokens:
        if tag[1] in set(['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']):
            newtags.append('v')
        elif tag[1] in set(['JJ', 'JJR', 'JJS']):
             newtags.append('a')
        elif tag[1] in set(['RB', 'RBR', 'RBS']):
             newtags.append('r')
        elif tag[1] in set(['NNS', 'NN', 'NNP', 'NNPS']):
             newtags.append('n')
        else:
             newtags.append('a')
    
    lem_words=[] #edw tha mpoun oi lematized lekseis pou exoume kratisei apo to preprocessing
    counter=0 #vazoume ton counter gia na kanoume iterate ta stoixeia tis listas twn tags
    for word in tagged_filtered_tokens:    
        lem_words.append(wnl.lemmatize(word[0],newtags[counter]))
        counter+=1
    lem_words = [word for word in lem_words if word not in stop] #svinoume ta stopwords
    return lem_words

def tokenizer_preproccessor_imdb(text):
    stop = nltk.corpus.stopwords.words('english')
    stop.append('film')
    stop.append('movie')
    stop.append('br')
    #kanoume preprocessing ta dedomena mas
    word_tokens=word_tokenize(text.lower()) #kanoume tokenize
    #print(word_tokens)
    #print("Έχουμε " + str(len(word_tokens))+ " tokens")
    #filtered_word_tokens = [word for word in word_tokens if word not in stop] #svinoume ta stopwords
    #print("Αφού αφαιρέσαμε τα stopwords, έχουμε τελικά " + str(len(filtered_word_tokens)) + " tokens")
    #print(filtered_word_tokens)
    filtered_word_tokens = [re.sub(r'[^A-Za-z]+', '', x) for x in word_tokens]  #svinoume ta punctuations
    filtered_word_tokens = [x for x in filtered_word_tokens if ((x.startswith("'")==0 or x.startswith('-') or x.startswith('.')))]  #svinoume ta punctuations

    #filtered_word_tokens = [x for x in filtered_word_tokens if x not in ['film','movie']]  #svinoume ta punctuations    
    filtered_word_tokens = [word for word in filtered_word_tokens if len(word)>1] #svinoume tis mikres lekseis
    #print(filtered_word_tokens)
    #print("Αφού αφαιρέσαμε τα σημεία στίξης και τις μικρές λέξεις, έχουμε τελικά " + str(len(filtered_word_tokens)) + " tokens")
    #print(filtered_word_tokens)
   
    tagged_filtered_tokens=nltk.pos_tag(filtered_word_tokens) #kanoume pos tag tis lekseis gia syntaktiki analysi
    #print(tagged_filtered_tokens)
            #ftiaxnoume ta pos tags wste na mpoun san input sto sentisynset
    #epeidh ta dedomena tou postag ginontai tuples, ta metatrepoume se lista
    newtags=[] #lista me ta nea tags kai idio counter me ta tagged words
    for tag in tagged_filtered_tokens:
        if tag[1] in set(['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']):
            newtags.append('v')
        elif tag[1] in set(['JJ', 'JJR', 'JJS']):
             newtags.append('a')
        elif tag[1] in set(['RB', 'RBR', 'RBS']):
             newtags.append('r')
        elif tag[1] in set(['NNS', 'NN', 'NNP', 'NNPS']):
             newtags.append('n')
        else:
             newtags.append('a')
    
    lem_words=[] #edw tha mpoun oi lematized lekseis pou exoume kratisei apo to preprocessing
    counter=0 #vazoume ton counter gia na kanoume iterate ta stoixeia tis listas twn tags
    for word in tagged_filtered_tokens:    
        lem_words.append(wnl.lemmatize(word[0],newtags[counter]))
        counter+=1
    lem_words = [word for word in lem_words if word not in stop] #svinoume ta stopwords
    #print(lem_words)
    
    return lem_words


# In[5]:


def SentimentAnalysis_Sentiwordnet(text_samples):
    my_sentiments=[] #h lista pou tha periexei tosynaisthitiko score kathe keimenou
    my_sentiments_class=[]
    #print(stopwords)
    stop = set(stopwords.words('english'))
    for text in text_samples:
        #kanoume preprocessing ta dedomena mas
        word_tokens=word_tokenize(text.lower()) #kanoume tokenize
        #print("Έχουμε " + str(len(word_tokens))+ " tokens")
        filtered_word_tokens = [word for word in word_tokens if word not in stop] #svinoume ta stopwords
        #print("Αφού αφαιρέσαμε τα stopwords, έχουμε τελικά " + str(len(filtered_word_tokens)) + " tokens")
        #print(filtered_word_tokens)
        filtered_word_tokens = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in filtered_word_tokens]  #svinoume ta punctuations
        filtered_word_tokens = [word for word in filtered_word_tokens if len(word)>1] #svinoume tis mikres lekseis
        #print("Αφού αφαιρέσαμε τα σημεία στίξης και τις μικρές λέξεις, έχουμε τελικά " + str(len(filtered_word_tokens)) + " tokens")
        #print(filtered_word_tokens)
        tagged_filtered_tokens=nltk.pos_tag(filtered_word_tokens) #kanoume pos tag tis lekseis gia syntaktiki analysi
        #print(tagged_filtered_tokens)
                #ftiaxnoume ta pos tags wste na mpoun san input sto sentisynset
        #epeidh ta dedomena tou postag ginontai tuples, ta metatrepoume se lista
        newtags=[] #lista me ta nea tags kai idio counter me ta tagged words
        for tag in tagged_filtered_tokens:
            if tag[1] in set(['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']):
                newtags.append('v')
            elif tag[1] in set(['JJ', 'JJR', 'JJS']):
                 newtags.append('a')
            elif tag[1] in set(['RB', 'RBR', 'RBS']):
                 newtags.append('r')
            elif tag[1] in set(['NNS', 'NN', 'NNP', 'NNPS']):
                 newtags.append('n')
            else:
                 newtags.append('a')
                    
                    

        lem_words=[] #edw tha mpoun oi lematized lekseis pou exoume kratisei apo to preprocessing
        counter=0 #vazoume ton counter gia na kanoume iterate ta stoixeia tis listas twn tags
        for word in tagged_filtered_tokens:    
            lem_words.append(wnl.lemmatize(word[0],newtags[counter]))
            counter+=1
           # print (newtags)
       # new_words_tags_dict = {'word':'synscore'}
        #print(newtags,lem_words)
            #print(lem_words)
        #ypologizoume to synaisthima kathe leksis , mazi me to POSTAG tis
        posscore=0
        negscore=0
        for i in range(len(lem_words)): 
            synsets = swn.senti_synsets(lem_words[i],newtags[i])
            for synst in synsets: #athroizoume ta thetika kai ta arnhtika score kathe leksis
                posscore=posscore+synst.pos_score()
                negscore=negscore+synst.neg_score()     
        my_sentiments.append(posscore-negscore)
        #print(my_sentiments)
        if (posscore-negscore)>=0:
            my_sentiments_class.append(1)
        else:
            my_sentiments_class.append(0)
    print(len(my_sentiments))
    return(my_sentiments_class)


# In[6]:


def SentimentAnalysis_Vader(text_samples):
    analyzer = SentimentIntensityAnalyzer()
    vader_sentiments=[]
    vader_class_sentiments=[]
    for text in text_samples:
        sum=0
        #Kovoume kathe keimeno se protaseis wste na vgalei sentiment polarity o vader, ta opoia athroizoume
        sentences=text.split('\n')   
        for sentence in sentences:
            sent = analyzer.polarity_scores(sentence)
            #print("{:-<65} {}".format(sentence, vs))
            sum=sum+sent['compound']
        average=sum/len(sentences)
        vader_sentiments.append(average)
        if (average>=0):
            vader_class_sentiments.append(1)       
        else:
            vader_class_sentiments.append(0)
        #print(sentiments)
        #print(average)
    #Data Examination for Unnotated Dataset
    print("Έχουμε " + str(vader_class_sentiments.count(1)) + " χαρούμενα τραγούδια")
    print("Έχουμε " + str(vader_class_sentiments.count(0)) + " στενάχωρα τραγούδια")
    return vader_class_sentiments


# In[ ]:


def classifier_finder(X_train,X_test,y_train,y_test):
    classifiers=[]

    scores=[]
    model1=LogisticRegression(max_iter=1000)
    classifiers.append(model1)
    model2=MultinomialNB()
    classifiers.append(model2)
    model4 = tree.DecisionTreeClassifier()
    classifiers.append(model4)
    model5 = RandomForestClassifier()
    classifiers.append(model5)
    model6=LinearSVC(max_iter=2000)
    classifiers.append(model6)
    
    
    for clf in classifiers:
        clf.fit(X_train, y_train)
        
        cvec_score = cross_val_score(clf, X_train, y_train, cv=10 )
        
        
        print("Η επιτυχία του  %s είναι:  %s"%(clf, cvec_score.mean()))
        scores.append(cvec_score.mean())
# DataFrame Accuracy 
    scores_df = pd.DataFrame()
    scores_df['params']= ['Logistisc Regression','Multinomial Naive Bayes','Decision Tree','Random Forest','Linear SVC']
    scores_df['scores']= scores
    print(scores_df)


# In[ ]:


def classifier_finder_music(X_train,y_train):
    classifiers=[]
    #COUNT VECTORIZER

    model1=LogisticRegression()
    classifiers.append(model1)
    model2 = tree.DecisionTreeClassifier()
    classifiers.append(model2)
    model3 = RandomForestClassifier()
    classifiers.append(model3)
    model4=LinearSVC()
    classifiers.append(model4)
    model5 = KNeighborsClassifier()
    classifiers.append(model5)
    scorelist=[]

    for clf in classifiers:
        clf.fit(X_train, y_train)
        cvec_score = cross_val_score(clf, X_train, y_train, cv=10)
        print("Τα αποτελέσματα του Cross-Validation για τον {} είναι:  {} ".format(clf,cvec_score.mean()))                                      
        scorelist.append(cvec_score.mean())
# DataFrame Accuracy 
    scores_df = pd.DataFrame()
    scores_df['params']= ['Logistisc Regression','Decision Tree','Random Forest','Linear SVC','K-nn']
    scores_df['scores']= scorelist
    print(scores_df)
    


# In[1]:


#δημιουργούμε συναρτήσεις για να αποθηκεύουμε και να φορτώνουμε  τις λίστες που θα φτιάξουμε με την ανάλυση του vader 
#και του sentiwordnet
def saveList(myList,filename):
    np.save(filename,myList)
    print("Saved successfully!")
    
def loadList(filename):
    tempNumpyArray=np.load(filename,allow_pickle=True)
    return tempNumpyArray.tolist()


# In[ ]:






[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dtchmnt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dtchmnt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dtchmnt\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dtchmnt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\dtchmnt\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dtchmnt\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexi

In [12]:
# # φορτώνουμε τους classifiers που φτιάξαμε
imdb_clf = joblib.load('imdb_clf.pkl')
sentiwordnet_clf = joblib.load('sentiwordnet_clf.pkl')
vader_clf = joblib.load('vader_clf.pkl')
#φορτωνουμε τους vectorizers που φτιάξαμε, ώστε να κάνουμε transform τα X_test με βαση αυτα τα vects
vader_cvec = joblib.load('vader_cvec.pkl')
sentiwordnet_cvec = joblib.load('sentiwordnet_cvec.pkl')
imdb_tvec=joblib.load('imdb_tvec.pkl')

C:\Users\dtchmnt\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\dtchmnt\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [13]:
#φορτώνουμε τα σετ ε΄λέγχου 
X_test_sentiwordnet=loadList("X_test_sentiwordnet.npy")
y_test_sentiwordnet=loadList("y_test_sentiwordnet.npy")
X_test_imdb=loadList("X_test_imdb.npy")
y_test_imdb=loadList("y_test_imdb.npy")
X_test_vader=loadList("X_test_vader.npy")
y_test_vader=loadList("y_test_vader.npy")

In [14]:
X_vader_by_vader_dtm=loadList('X_test_vader_dtm.npy')
X_sentiwordnet_by_sentiwordnet_dtm=loadList('X_test_sentiwordnet_dtm.npy')
X_imdb_by_imdb_dtm=loadList('X_test_imdb_dtm.npy')
#kanoume transform ta ypoloipa dianysmata, wste na mporoume na kanoume predictions gia to ekastote X_test
#gia ton vect tou vader
X_sentiwordnet_by_vader_dtm = vader_cvec.transform(X_test_sentiwordnet)
X_imdb_by_vader_dtm = vader_cvec.transform(X_test_imdb)
#gia ton vect tou sentiwordnet
X_vader_by_sentiwordnet_dtm = sentiwordnet_cvec.transform(X_test_vader)
X_imdb_by_sentiwordnet_dtm = sentiwordnet_cvec.transform(X_test_imdb)
#gia ton vect tou imdb
X_vader_by_imdb_dtm = imdb_tvec.transform(X_test_vader)
X_sentiwordnet_by_imdb_dtm = imdb_tvec.transform(X_test_sentiwordnet)


In [22]:
#kanoume provlepsh gia to kathe dataset me vash ton classifier tou Vader
#ta onomata twn metavlitwn pou apothikevoume, anaferontai prwta sto dataset gia to opoio dokimazoume, kai meta gia 
#ton classifier. Ara to vader_score_vader anaferetai ston vader classifier pou tha prospathisei na kanei predictions gia to vader
#dataset. Antistoixa, to sentiwordnet_score_vader, shmainei pws o vader classifier tha prospathisei na kanei predictions gia to
#set dedomenwn apo to sentiwordnet
ypred_for_vader = vader_clf.predict(X_vader_by_vader_dtm)
vader_score_vader=accuracy_score(y_test_vader, ypred_for_vader)

ypred_for_sentiwordnet =vader_clf.predict(X_sentiwordnet_by_vader_dtm)
sentiwordnet_score_vader = accuracy_score(y_test_sentiwordnet, ypred_for_sentiwordnet)

ypred_for_imdb =vader_clf.predict(X_imdb_by_vader_dtm)
imdb_score_vader=accuracy_score(y_test_imdb,ypred_for_imdb)
print("Η επιτυχία του ταξινομητή του Vader για το σετ δεδομένων Vader είναι {}".format(vader_score_vader))
print("Η επιτυχία του ταξινομητή του Vader για το σετ δεδομένων SentiWordNet είναι {}".format(sentiwordnet_score_vader))
print("Η επιτυχία του ταξινομητή του Vader για το σετ δεδομένων Imdb είναι {}".format(imdb_score_vader))
print("Πολλαπλασιάζουμε τα σκορ που βρήκαμε")
print("Μέσος Όρος: ",(vader_score_vader+sentiwordnet_score_vader+imdb_score_vader)/3)
print("Μέσος Όρος μόνο για στίχους: ",(vader_score_vader+sentiwordnet_score_vader)/2)
print()
#kanoume provlepsh gia to kathe dataset me vash ton classifier tou SentiWordNet
ypred_for_vader = sentiwordnet_clf.predict(X_vader_by_sentiwordnet_dtm)
vader_score_sentiwordnet=accuracy_score(y_test_vader, ypred_for_vader)

ypred_for_sentiwordnet =sentiwordnet_clf.predict(X_sentiwordnet_by_sentiwordnet_dtm)
sentiwordnet_score_sentiwordnet = accuracy_score(y_test_sentiwordnet, ypred_for_sentiwordnet)

ypred_for_imdb =sentiwordnet_clf.predict(X_imdb_by_sentiwordnet_dtm)
imdb_score_sentiwordnet=accuracy_score(y_test_imdb,ypred_for_imdb)
print("Η επιτυχία του ταξινομητή του SentiWordNet για το σετ δεδομένων Vader είναι {}".format(vader_score_sentiwordnet))
print("Η επιτυχία του ταξινομητή του SentiWordNet για το σετ δεδομένων SentiWordNet είναι {}".format(sentiwordnet_score_sentiwordnet))
print("Η επιτυχία του ταξινομητή του SentiWordNet για το σετ δεδομένων Imdb είναι {}".format(imdb_score_sentiwordnet))
print("Πολλαπλασιάζουμε τα σκορ που βρήκαμε")
print("Μέσος Όρος: ",(vader_score_sentiwordnet+sentiwordnet_score_sentiwordnet+imdb_score_sentiwordnet)/3)
print("Μέσος Όρος μόνο για στίχους: ",(vader_score_sentiwordnet+sentiwordnet_score_sentiwordnet)/2)

print()

#kanoume provlepsh gia to kathe dataset me vash ton classifier tou imdb
ypred_for_vader = imdb_clf.predict(X_vader_by_imdb_dtm)
vader_score_imdb=accuracy_score(y_test_vader, ypred_for_vader)

ypred_for_sentiwordnet =imdb_clf.predict(X_sentiwordnet_by_imdb_dtm)
sentiwordnet_score_imdb = accuracy_score(y_test_sentiwordnet, ypred_for_sentiwordnet)

ypred_for_imdb =imdb_clf.predict(X_imdb_by_imdb_dtm)
imdb_score_imdb=accuracy_score(y_test_imdb,ypred_for_imdb)
print("Η επιτυχία του ταξινομητή του Imdb για το σετ δεδομένων Vader είναι {}".format(vader_score_imdb))
print("Η επιτυχία του ταξινομητή του Imdb για το σετ δεδομένων SentiWordNet είναι {}".format(sentiwordnet_score_imdb))
print("Η επιτυχία του ταξινομητή του Imdb για το σετ δεδομένων Imdb είναι {}".format(imdb_score_imdb))
print("Πολλαπλασιάζουμε τα σκορ που βρήκαμε")
print("Μέσος Όρος: ",(vader_score_imdb+sentiwordnet_score_imdb+imdb_score_imdb)/3)
print("Μέσος Όρος μόνο για στίχους: ",(vader_score_imdb+sentiwordnet_score_imdb)/2)
print()


Η επιτυχία του ταξινομητή του Vader για το σετ δεδομένων Vader είναι 0.8638334778837814
Η επιτυχία του ταξινομητή του Vader για το σετ δεδομένων SentiWordNet είναι 0.7676496097137901
Η επιτυχία του ταξινομητή του Vader για το σετ δεδομένων Imdb είναι 0.6371
Πολλαπλασιάζουμε τα σκορ που βρήκαμε
Μέσος Όρος:  0.7561943625325238
Μέσος Όρος μόνο για στίχους:  0.8157415437987858

Η επιτυχία του ταξινομητή του SentiWordNet για το σετ δεδομένων Vader είναι 0.7393755420641804
Η επιτυχία του ταξινομητή του SentiWordNet για το σετ δεδομένων SentiWordNet είναι 0.9300086730268864
Η επιτυχία του ταξινομητή του SentiWordNet για το σετ δεδομένων Imdb είναι 0.5945
Πολλαπλασιάζουμε τα σκορ που βρήκαμε
Μέσος Όρος:  0.7546280716970223
Μέσος Όρος μόνο για στίχους:  0.8346921075455334

Η επιτυχία του ταξινομητή του Imdb για το σετ δεδομένων Vader είναι 0.6427580225498699
Η επιτυχία του ταξινομητή του Imdb για το σετ δεδομένων SentiWordNet είναι 0.6480485689505637
Η επιτυχία του ταξινομητή του Imdb για το σε